In [505]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline

In [506]:
matches=pd.read_csv('matches.csv')
delivery=pd.read_csv('deliveries.csv')

In [507]:
matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [508]:
delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [509]:
total_score_df=delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

In [510]:
total_score_df=total_score_df[total_score_df['inning']==1]

In [511]:
total_score_df.head()

,match_id,inning,total_runs
0,1,1,207
2,2,1,184
4,3,1,183
6,4,1,163
8,5,1,157


In [512]:
new_df=matches.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [513]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [514]:
new_df['team1']=new_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
new_df['team2']=new_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

In [515]:
new_df.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,184
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,3,183
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,4,163
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,157


In [516]:
new_df['team1']=new_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
new_df['team2']=new_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [517]:
new_df=new_df[new_df['team1'].isin(teams)]
new_df=new_df[new_df['team2'].isin(teams)]

In [518]:
new_df.shape

(641, 20)

In [519]:
new_df=new_df[new_df['dl_applied']==0]

In [520]:
new_df=new_df[['match_id','city','winner','total_runs']]

In [521]:
new_df.head()

,match_id,city,winner,total_runs
0,1,Hyderabad,Sunrisers Hyderabad,207
4,5,Bangalore,Royal Challengers Bangalore,157
6,7,Mumbai,Mumbai Indians,178
7,8,Indore,Kings XI Punjab,148
9,10,Mumbai,Mumbai Indians,158


In [522]:
new_df2=new_df.merge(delivery,on='match_id')

In [523]:
new_df2.head()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
0,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,Hyderabad,Sunrisers Hyderabad,207,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [524]:
new_df2=new_df2[new_df2['inning']==2]

In [525]:
new_df4=new_df2[['match_id','total_runs_y']]
new_df2['current_score'] = new_df4.groupby('match_id').cumsum()['total_runs_y']


In [526]:
new_df2.tail()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score
149573,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,1,0,1,NaN,NaN,NaN,152
149574,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,2,0,2,NaN,NaN,NaN,154
149575,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,1,0,1,SR Watson,run out,KH Pandya,155
149576,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,2,0,2,NaN,NaN,NaN,157
149577,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,6,SN Thakur,...,0,0,0,0,0,0,SN Thakur,lbw,NaN,157


In [527]:
new_df2['runs_left']=new_df2['total_runs_x']-new_df2['current_score']
new_df2.tail()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left
149573,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,1,0,1,NaN,NaN,NaN,152,0
149574,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,2,0,2,NaN,NaN,NaN,154,-2
149575,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,1,0,1,SR Watson,run out,KH Pandya,155,-3
149576,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,2,0,2,NaN,NaN,NaN,157,-5
149577,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,6,SN Thakur,...,0,0,0,0,0,SN Thakur,lbw,NaN,157,-5


In [528]:
new_df2['balls_left'] = 126 - (new_df2['over']*6 + new_df2['ball'])

In [529]:
#wicketsleft
new_df3=new_df2[['match_id','player_dismissed']]
new_df3['player_dismissed'] = new_df3['player_dismissed'].fillna("0")
new_df3['player_dismissed'] = new_df3['player_dismissed'].apply(lambda x:x if x == "0" else "1")
new_df3['player_dismissed'] = new_df3['player_dismissed'].astype('int')
wickets= new_df3.groupby('match_id').cumsum()['player_dismissed'].values
new_df2['wickets_left'] = 10 - wickets
new_df2.tail()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26484\2597816318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['player_dismissed'] = new_df3['player_dismissed'].fillna("0")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26484\2597816318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['player_dismissed'] = new_df3['player_dismissed'].apply(lambda x:x if x == "0" else "1")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26484\2597816318.py:5: SettingWithCopyWarning: 
A value is trying to be 

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets_left
149573,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,1,0,1,NaN,NaN,NaN,152,0,4,5
149574,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,2,0,2,NaN,NaN,NaN,154,-2,3,5
149575,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,1,0,1,SR Watson,run out,KH Pandya,155,-3,2,4
149576,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,2,0,2,NaN,NaN,NaN,157,-5,1,4
149577,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,6,SN Thakur,...,0,0,0,SN Thakur,lbw,NaN,157,-5,0,3


In [530]:
# new_df2['wickets_left']=10-new_df2['wickets']
new_df2['balls_left']

125       119
126       118
127       117
128       116
129       115
         ... 
149573      4
149574      3
149575      2
149576      1
149577      0
Name: balls_left, Length: 72413, dtype: int64

In [531]:
new_df2['crr'] = (new_df2['current_score']*6)/(120 - new_df2['balls_left'])
new_df2.describe()

,match_id,total_runs_x,inning,over,ball,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,current_score,runs_left,balls_left,wickets_left,crr
count,72413.000000,72413.000000,72413.0,72413.000000,72413.000000,72413.0,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000,72413.000000
mean,2081.518747,165.583956,2.0,9.955726,3.608565,0.0,0.036692,0.005054,0.021267,0.004101,0.000138,1.236214,0.067253,1.303468,73.325577,92.258379,62.657078,7.535967,7.439523
std,3724.771739,29.282200,0.0,5.563866,1.806566,0.0,0.245309,0.122473,0.199127,0.074674,0.026277,1.603526,0.344512,1.603395,46.906255,50.021962,33.404593,2.138637,2.275850
min,1.000000,65.000000,2.0,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.000000,-2.000000,0.000000,0.000000
25%,192.000000,146.000000,2.0,5.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.000000,53.000000,35.000000,6.000000,6.257143
50%,396.000000,165.000000,2.0,10.000000,4.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,70.000000,92.000000,63.000000,8.000000,7.480519
75%,569.000000,185.000000,2.0,15.000000,5.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,109.000000,130.000000,92.000000,9.000000,8.682353
max,11415.000000,250.000000,2.0,20.000000,9.000000,0.0,5.000000,4.000000,5.000000,5.000000,5.000000,7.000000,7.000000,10.000000,223.000000,249.000000,119.000000,10.000000,42.000000


In [532]:
new_df2 = new_df2[new_df2['balls_left'] != 0]
new_df2['rrr'] = (new_df2['runs_left']*6)/new_df2['balls_left']
new_df2.describe()

,match_id,total_runs_x,inning,over,ball,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,current_score,runs_left,balls_left,wickets_left,crr,rrr
count,72172.000000,72172.000000,72172.0,72172.000000,72172.000000,72172.0,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000,72172.000000
mean,2082.046389,165.542648,2.0,9.922186,3.600579,0.0,0.036690,0.004988,0.021310,0.004101,0.000139,1.236006,0.067228,1.303234,73.040902,92.501746,62.866305,7.551377,7.437890,10.349743
std,3725.285080,29.286095,0.0,5.542740,1.804278,0.0,0.245474,0.122228,0.199389,0.074706,0.026321,1.603241,0.344712,1.603214,46.703971,49.916667,33.263185,2.123458,2.278411,13.659793
min,1.000000,65.000000,2.0,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.000000,-2.000000,0.000000,0.000000,-510.000000
25%,192.000000,146.000000,2.0,5.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.000000,53.000000,35.000000,6.000000,6.250000,7.135135
50%,396.000000,165.000000,2.0,10.000000,4.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,69.000000,92.000000,64.000000,8.000000,7.478261,8.869565
75%,569.000000,185.000000,2.0,15.000000,5.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,108.000000,130.000000,92.000000,9.000000,8.682353,10.879341
max,11415.000000,250.000000,2.0,20.000000,9.000000,0.0,5.000000,4.000000,5.000000,5.000000,5.000000,7.000000,7.000000,10.000000,222.000000,249.000000,119.000000,10.000000,42.000000,678.000000


In [533]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [534]:
new_df2['result'] = new_df2.apply(result,axis=1)

In [535]:
final_df = new_df2[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]

In [536]:
final_df = final_df.sample(final_df.shape[0])

In [537]:
final_df.isnull().sum()/final_df.shape[0]*100

batting_team    0.00000
bowling_team    0.00000
city            1.15003
runs_left       0.00000
balls_left      0.00000
wickets_left    0.00000
total_runs_x    0.00000
crr             0.00000
rrr             0.00000
result          0.00000
dtype: float64

In [538]:
final_df.dropna(inplace=True)

In [539]:
final_df = final_df[final_df['balls_left'] != 0]

In [540]:
X = final_df.drop('result', axis=1)
y = final_df['result']
numeric_features = ['runs_left','balls_left','wickets_left','total_runs_x','crr','rrr']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

In [541]:
categorical_features = ['batting_team','bowling_team','city']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [542]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [543]:
from sklearn.linear_model import LogisticRegression

In [546]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LogisticRegression(solver='liblinear'))
])

In [544]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [545]:
X_train

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr
145179,Chennai Super Kings,Sunrisers Hyderabad,Chennai,11,15,7,179,9.600000,4.400000
79710,Kings XI Punjab,Mumbai Indians,Mumbai,142,93,8,174,7.111111,9.161290
58082,Mumbai Indians,Royal Challengers Bangalore,Chennai,104,58,7,185,7.838710,10.758621
96176,Delhi Daredevils,Rajasthan Royals,Ahmedabad,97,20,2,201,6.240000,29.100000
7216,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,184,73,5,222,4.851064,15.123288
...,...,...,...,...,...,...,...,...,...
2419,Kolkata Knight Riders,Delhi Daredevils,Delhi,1,2,4,168,8.491525,3.000000
6977,Mumbai Indians,Kolkata Knight Riders,Bangalore,76,94,8,107,7.153846,4.851064
122409,Sunrisers Hyderabad,Rajasthan Royals,Hyderabad,33,54,9,133,9.090909,3.666667
148733,Mumbai Indians,Chennai Super Kings,Chennai,128,112,9,136,6.000000,6.857143


In [190]:
X_train.isnull().sum()

batting_team    0
bowling_team    0
city            0
runs_left       0
balls_left      0
wickets_left    0
total_runs_x    0
crr             0
rrr             0
dtype: int64

In [191]:
# model.fit(X_train, y_train)

In [547]:
X.describe()

,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr
count,71342.000000,71342.000000,71342.000000,71342.000000,71342.000000,71342.000000
mean,92.621065,62.878725,7.553251,165.726963,7.447791,10.368308
std,49.957572,33.259471,2.124334,29.294967,2.280498,13.726365
min,-16.000000,-2.000000,0.000000,65.000000,0.000000,-510.000000
25%,53.000000,35.000000,6.000000,147.000000,6.265782,7.151515
50%,92.000000,64.000000,8.000000,165.000000,7.500000,8.880000
75%,130.000000,92.000000,9.000000,185.000000,8.696385,10.894737
max,249.000000,119.000000,10.000000,250.000000,42.000000,678.000000


In [548]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['runs_left', 'balls_left',
                                                   'wickets_left',
                                                   'total_runs_x', 'crr',
                                                   'rrr']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('regressor', LogisticRegression(solver='liblinear'))])

In [549]:
y_pred = model.predict(X_test)

In [550]:
from sklearn.metrics import accuracy_score

In [551]:
accuracy_score(y_test, y_pred)

0.7994954096292662

In [552]:
import pickle
pickle.dump(model,open('pipe.pkl','wb'))